# Insegnare a un computer a riconoscere immagini

## Ma prima di tutto...

Esegui la cella qui sotto, senza preoccuparti più di tanto di che cosa succede: serve solo a rendere disponibile del codice che abbiamo scritto.

In [ ]:
try:
    import ice
except ModuleNotFoundError:
    import sys
    if "google.colab" in sys.modules:
        !python -m pip install -qqq --upgrade -- uv && python -m uv pip install --system --quiet -- https://github.com/baggiponte/aiss-2024.git
        !mkdir public && curl -L https://github.com/baggiponte/aiss-2024/raw/main/public/02-nn_training.mp4 -o public/02-nn_training.mp4

# MNIST: un dataset fatto di cifre scritte a mano

Cominciamo con uno dei dataset più famosi della storia delle reti neurali. [MNIST](https://en.wikipedia.org/wiki/MNIST_database) è un dataset di settantamila cifre da 0 a 9 scritte a mano e digitalizzate in un formato da 28x28 pixel, in bianco e nero (scala di grigio). Viene usato in un articolo piuttosto famoso, firmato da diversi "padri" del deep learning. Per chi fosse interessato, [qui](https://www.youtube.com/watch?v=mTtDfKgLm54&list=PLLHTzKZzVU9e6xUfG10TkTWApKSZCzuBI) c'è la prima lezione di un corso di Deep Learning della New York University: forse non è il caso di seguirlo tutto, ma se ve la cavate con l'inglese potete sentire Yann LeCunn, uno degli autori del paper di MNIST e attuale Chief AI Scientist di Meta (cioè il capo di tutta la parte di AI di Facebook, Instagram, ecc), raccontare la storia del deep learning.

Useremo anche noi questo dataset per allenare la nostra prima rete neurale. Fun fact: il tasso d'errore più basso raggiunto per MNIST è circa dello 0.2/0.3%. Quanto riusciremo ad avvicinarci?

Prima di costruire, esegui il contenuto della cella seguente e usa lo slider per esplorare le fotografie. Potrebbero esserci delle foto etichettate male: alcune fonti riportano che siano almeno 4 (che può essere il motivo per cui, di fatto, è impossibile ottenere il 100% di accuratezza, a meno di correzioni).

In [ ]:
from ice.utils import load_mnist, display

train, test = load_mnist(path="../data")

display(train)

L'immagine che vediamo è semplicemente una matrice di numeri: possiamo vederlo così:

In [ ]:
# questa è un po' di magia nera, ma non è niente di difficile
# nota che `[i]` viene usato per accedere all'elemento i-esimo di una lista/contenitore
image, label = train[0]

image

Vista così la forma non ha molto senso (solo perché la matrice non è allineata in 28x28), ma si nota molto bene di che cosa è fatta: una grossa matrice con valori che vanno da 0 (nero) a 255 (bianco).

Già a occhio nudo, comunque, possiamo vedere dei pattern: il processo di allenamento di una rete neurale è una metodo che abbiamo affinato per far sì che un computer impari a riconoscerli e sfruttarli per associare un input a un determinato output. Vediamo come!

# La struttura di una rete neurale

Prima di mettere mano alla tastiera, però, proviamo a spiegare un po' come funzionano le reti neurali.

## Come funzionano le reti neurali?

Non scendiamo nel dettaglio di come funziona una rete neurale: non perché sia difficile, ma perché sono necessari alcuni concetti. In linea di massima, nel caso di un problema di classificazione come il nostro, una rete neurale fa quello che si vede in [questo spezzone](https://youtu.be/mTtDfKgLm54?si=STy0SyJe17i73_Ag&t=2809) della lezione che abbiamo condiviso sopra.

Si comincia da dei punti nello spazio che non sono *linearmente separabili*, cioè non si possono separare con una o più linee rette:

<div style="text-align: center;">
    <img src="../public/01-blob.png" style="width: 300px; height: auto;">
</div>

L'algoritmo "apprende" quali sono le trasformazioni dello spazio (distorsioni, traslazioni, e simili) per isolare sottoinsiemi nei dati, finché non diventano separabili con una linea.

In [ ]:
from IPython.display import Video

Video("./public/02-nn_training.mp4", width=500)

In altre parole, semplificando un po', è come se il modello dovesse imparare a tracciare delle linee storte:

<div style="text-align: center;">
    <img src="../public/03-blob_separated.png" style="width: 300px; height: auto;">
</div>

In che senso "impara"? In parole povere, comincia con delle trasformazioni a caso, genera una predizione, giusta o sbagliata, e riceve un "feedback" a ogni iterazione su come aggiustare il tiro. A un certo punto, quando l'errore è accettabile, interrompiamo l'allenamento.

Se vi ricordate l'equazione della retta: 

$y = mx + q$

Ecco, una rete neurale non ha solo un parametro $m$ e $q$, ma ne ha solitamente da milioni a miliardi. L'obiettivo dell'allenamento è, partendo da $x$, trovare dei parametri (detti anche pesi) abbastanza precisi da trovare la $y$ con un margine d'errore accettabile. Come funziona "l'aggiustamento"? È una procedura *iterativa*, cioè ripetuta:

1. usando i pesi attuali si genera una predizione;
2. si misura l'errore della predizione;
3. infine si usa l'errore per modificare leggermente i pesi;

E via daccapo dal punto 1. Per chi l'avesse visto, la procedura più semplice - e quella che useremo noi - è simile al metodo di Newton. In altre parole, si usano le derivate.

Ci sono molte introduzioni ben fatte (praticamente solo in inglese). Qui una carrellata:

1. [MLU Explain](https://mlu-explain.github.io/neural-networks/). Spiegazione completa e interattiva.
2. [3Blue1Brow](https://www.youtube.com/watch?v=aircAruvnKk&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi). Sicuramente più densa ma non per questo difficile da capire. Tenetelo come riferimento per vari spiegoni di matematica (per il quinto anno ma anche per l'università): non solo spiega bene, ma fa anche capire il significato e come interpretare certi concetti di matematica, probabilità e statistica.
3. Con un po' più di dettaglio [introduzione](https://www.youtube.com/watch?v=UOvPeC8WOt8) e poi [parte 2](https://www.youtube.com/watch?v=-at7SLoVK_I)

Lasciamoci alle spalle la teoria, e andiamo alla pratica.

## Disegnare una rete neurale

Per quanto semplice, la spiegazione delle reti neurali deve essere già stata un po' densa. Per fortuna, in Python non dobbiamo occuparci noi di programare niente di tutto questo! `pytorch` è una libreria (o *framework*) che ci permette di disegnare e allenare reti neurali rapidamente.

Basta una decina di righe per definirne una:

In [ ]:
import torch.nn as nn

class NeuralNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layer = nn.Linear(28 * 28, 10)

    def forward(self, x):
        x_flat = self.flatten(x)
        logits = self.layer(x_flat)
        return logits

model = NeuralNetwork()

Ci sono comunque un po' di cose da spacchettare, quindi andiamo con ordine.

Innanzitutto: che cosa c'è scritto? Stiamo definendo una *classe*, cioè creiamo un particolare tipo di oggetto. Questo oggetto *eredita* da `nn.Module`. Un oggetto - lo abbiamo visto nell'introduzione - è un contenitore di "attributi" e "comportamenti". Per fare un esempio: un oggetto `Cane` potrebbe avere come "attributi" il suo nome, il colore, il cibo preferito... E come "comportamenti" (detti anche *metodi*) abbaiare, rincorrere, recuperare un bastone. Non ci serve sapere molto altro degli oggetti, né come crearli, salvo tenere presente che per costurire una rete funzionante dobbiamo definire due cose:

1. La struttura della nostra rete, che facciamo nel metodo `__init__`
2. Come generare una previsione, con il metodo `forward`.

In altre parole, tutto quello su cui dobbiamo concentrarci è:

```python
        self.flatten = nn.Flatten()
        self.layer = nn.Linear(28 * 28, 10)
```

Questa è la struttura della nostra rete (gli "strati" del "panino"). Sono degli attributi che abbiamo chiamato `flatten` e `layer`.

1. Il primo "strato" (in inglese, *layer*) che appiattisce (*flatten*) la fotografia da una matrice 28*28 a un'unico vettore di lunghezza 784. Non dobbiamo modificarlo.
2. Il secondo e ultimo strato è un oggetto particolare, `nn.Linear()`. Di fatto, è un modo compatto per scrivere i coefficienti del modello, un po' come se fosse `$y = mx + q$`. `nn.Linear(28 * 28, 10)` prende tutti i nostri pixel di input e li "rimappa" direttamente a 10 valori. In altre parole, è come se disegnassimo una retta che prende i nostri pixel e cerca di restituire un vettore che ci dice qual è il numero più probabile.
3. Nel `forward` step vediamo proprio come viene usata la rete neurale per generare le predizioni (dette *logits*). È un'altro pezzo che non dobbiamo modificare.

Possiamo vedere direttamente come è fatto questo oggetto:

In [ ]:
model

Questa rete neurale, però, non funzionerà molto bene. È molto semplice e le manca un elemento importante: la possibilità di imparare trasformazioni *non* lineari. Per questo aggiungiamo al panino uno strato "speciale":

In [ ]:
class MLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten(start_dim=1)
        self.stack = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x_flat = self.flatten(x)
        logits = self.stack(x_flat)
        return logits

model = MLP()

Per comodità, usiamo `nn.Sequential` per raggruppare i layer, ma di per sé non fa niente di speciale. Ora la rete ha tre strati:

1. Una mappatura da 784 dimensioni in input a una "intermedia" di 128 elementi - i cosiddetti "neuroni".
2. Uno strato intermedio che trasforma lo spazio in modo non lineare.
3. L'ultimo strato, dello stesso tipo del primo, che mappa da 128 a 10 neuroni - il nostro output.

In [ ]:
model

# Allenare un modello

Eccoci nella parte complicata - per il computer, però. Abbiamo scritto per te due oggetti, un `TrainerConfig` e un `Trainer`, per semplificarti il lavoro.

* `Trainer` è un oggetto che si occupa di allenare il modello per te e di dirti come sta procedendo l'allenamento.
* `TrainerConfig` contiene le configurazioni del `Trainer`.

Per cominciare, esegui il codice qui sotto. Quando il `trainer` ha finito di allenare il modello, qual è l'accuratezza media alla fine dell'allenamento?

In [ ]:
from ice.trainer import TrainerConfig, Trainer

config = TrainerConfig(
    epochs=3,
    batch_size=16,
    seed=42
)

In [ ]:
trainer = Trainer(model=model, config=config, dataset=train)

In [ ]:
trainer.fit()

Come vedete, però, le prestazioni lasciano a desiderare. Forse è il caso di fare di più: pensi di potercela fare? Qualche consiglio:

Modificare la rete neurale:

1. Attenzione, però: non cambiare mai il numero di dimensioni iniziale (`28 * 28`) e finale (`10`), altrimenti il modello non potrà più prendere in input le immagini di MNIST, o assegnare loro la categoria.
2. Alterna sempre un `nn.Linear(...)` con `nn.ReLU()`. Puoi anche usare `nn.Softmax()` o `nn.Tanh()`.
3. Puoi mettere tutti i neuroni che vuoi negli strati intermedi, e anche un numero a piacere di strati. Più strati e neuroni ci sono, più il modello ci metterà tempo ad allenare, anche se avrà migliori prestazioni.
4. Non ci sono regole per il numero di neuroni: puoi provare tanti strati con lo stesso numero, o ridurli progressivamente, o ancora prima accrescere e poi ridurre ancora. Puoi mettere più neuroni dei 784 input.
5. Per consuetudine, usiamo numeri che sono potenze di 2 (64, 128, 256, 512, ecc) perché così il computer riesce a fare le moltiplicazioni più velocemente, e quindi il modello si allena prima. Potete cambiare di poco (510, 520...) ma consigliamo di non provare numeri troppo strani.

Modificare il parametri del Trainer:
1. Allenare per più epoche.
2. Cambiare le dimensioni della `batch_size`. Di solito le batch non sono troppo grandi (solitamente non più di 128) ma potete provare tutte le potenze di due nel mezzo. Più la batch è grande e più il modello si allena velocemente.
3. Cambiare il `learning_rate`. Attenzione: un learning rate più basso significa che il modello ci mette più tempo ad allenare.

# Come valutare la qualità di un modello?

I più attenti avranno notato che, quando abbiamo caricato i dati, abbiamo sia un `train` che un `test`. Si tratta di una pratica cruciale per chi fa machine learning: tenere da parte un pezzo del dataset e non usarlo per allenare il modello. Viene messo da parte, dimenticato e sotto chiave, finché il modello non è pronto. Si usa infine il modello per comparare le prestazioni su dati che non ha visto prima. Per cui: quando siete convinti che il vostro modello sia adatto, e solo allora, eseguite le celle qui sotto e vedete come è andata 😎

In [ ]:
from ice.eval import predict, evaluate

predict(model=model, dataset=test)

In [ ]:
print(f"Percentage: {evaluate(model=model, dataset=test):.2%}")

# E le foto degli animali?

Non ce ne siamo dimenticati! Ma è un problema che richiede molte risorse e modelli più sofisticati: praticamente è certo che le architetture che vi abbiamo mostrato non sono sufficienti. Qualche considerazione:

1. Sono foto a colori, e cioè hanno tre canali (RGB, e cioè rosso, verde e blu), mentre le cifre di MNIST erano in bianco e nero. In altre parole: a parità di dimensioni, hanno 3 volte tanti pixel. L'input layer del nostro modello, quindi, avrebbe 784 * 3 = 2352 neuroni.
2. In realtà, però, queste immagini hanno dimensioni più grandi. La più piccola è di 120 pixel di larghezza, la più grande circa quattromila. Per allenare un modello, dobbiamo per forza ridimensionare le immagini per avere tutte quante le stesse dimensioni. Si finisce, però, per forza di cose, a perdere informazioni. Ma anche con delle foto 122 * 122, avremmo un input layer di 44.652 neuroni. Quasi 100 volte più grande di quello che abbiamo allenato finora, e stiamo parlando solo dell'input layer!
3. Le foto sono solo 6000. Dovremmo tenerne da parte almeno un 20% per il test set, quindi lavoreremmo con 4800 foto. Il solo training set di MNIST ne ha 60000, 10 volte di più. Allenare un modello 3-4 ordini di grandezza più grande (100-1000 volte più grande) richiede di aumentare i dati di una quantità simile. Provate ad allenare lo stesso modello con solo 600 foto e vedete se le prestazioni sul test set sono le stesse: probabilmente sì (il dataset non è molto variegato) ma sarà difficile passare dal 90% al 92% di accuratezza.

Che cosa possiamo fare? Una soluzione c'è.